# Matt Manner, Connor Nickol, Josh Gen
## 3 Apr 23

Importing packages

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
#import torch

Importing data

In [2]:
players = pd.read_csv('players.csv')#'/content/drive/MyDrive/nfl-big-data-bowl-2023/players.csv')
plays = pd.read_csv('plays.csv')#'/content/drive/MyDrive/nfl-big-data-bowl-2023/plays.csv')
locations = pd.DataFrame()
for i in range(1,8):
    url = 'week'+str(i)+'.csv' #/content/drive/MyDrive/nfl-big-data-bowl-2023/week
    week_data = pd.read_csv(url)
    locations = pd.concat([locations, week_data])

In [3]:
games = pd.read_csv('games.csv')

In [4]:
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL
1,2021091200,2021,1,09/12/2021,13:00:00,ATL,PHI
2,2021091201,2021,1,09/12/2021,13:00:00,BUF,PIT
3,2021091202,2021,1,09/12/2021,13:00:00,CAR,NYJ
4,2021091203,2021,1,09/12/2021,13:00:00,CIN,MIN


In [5]:
games = games[['gameId','homeTeamAbbr']]

In [6]:
locations = locations.merge(games, on = 'gameId', how = 'left')

In [7]:
pff = pd.read_csv('pffScoutingData.csv')

In [8]:
locations['uniqueplayId'] = locations['gameId'].astype(str) + locations['playId'].astype(str)

Need to make an adjacency matrix (nxnxt), edge feature matrix (exfxt), node feature matrix (nxfxt)

In [9]:
positions = ['QB','FB-L', 'FB', 'FB-R', 'HB-R', 'HB-L', 'HB', 'LWR', 'RWR','SLWR','SRoWR',
             'SRiWR', 'SRWR','SLoWR', 'SLiWR','TE-R','TE-L','TE-oR', 'TE-iR',
             'TE-oL','TE-iL', 'LT', 'LG','C','RG','RT', 'LE','LEO',
             'DLT', 'NLT','NT','NRT','DRT','RE','REO','LLB', 'LOLB', 'LILB', 
             'MLB', 'RILB','RLB', 'ROLB', 'LCB', 'SCBiL','SCBoL', 'SCBL',
             'SCBoR', 'SCBiR', 'SCBR', 'SSR','RCB','FS', 'FSR', 'FSL', 'SSL',  'SS', 'football']

In [10]:
locations.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event',
       'homeTeamAbbr', 'uniqueplayId'],
      dtype='object')

## For the edge matrix, we need to figure out how to make it number of edges by number of features keeping track of who is connected to who

## Working on the feature matrix. Things to implement:
- Cut the play after 3? seconds
- Corner orientation or maybe all defensive player orientation
- Yard line, down and distance, score margin, quarter all possible additions
- Personel package on the field? 

In [11]:
## Standardizing the ball to always be the same spot

## Filter out some of the coverages
plays = plays[plays.pff_passCoverage != 'Bracket']
plays = plays[plays.pff_passCoverage != 'Goal Line']
plays = plays[plays.pff_passCoverage != 'Miscellaneous']
plays = plays[plays.pff_passCoverage != 'Prevent']
plays = plays[plays.pff_passCoverage != 'Red Zone']

plays['uniqueplayId'] = plays['gameId'].astype(str) + plays['playId'].astype(str)

# merge the locational dataset with the plays dataset to remove these coverages in locational dataset
locations = plays.merge(locations, on='uniqueplayId')

In [12]:
# Dropping all of the rows that occur before the snap of the ball
snap_time = locations.loc[locations['event'] == 'ball_snap',['uniqueplayId', 'frameId']].rename(columns = {'frameId':'ball_snap_frame'})
snap_time_unique = snap_time.drop_duplicates(subset = 'uniqueplayId')

In [13]:
locations = locations.merge(snap_time_unique, on = 'uniqueplayId', how = 'left')
locations

,gameId_x,playId_x,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,x,y,s,a,dis,o,dir,event,homeTeamAbbr,ball_snap_frame
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,TB,6.0
1,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,TB,6.0
2,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,TB,6.0
3,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,TB,6.0
4,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,TB,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891830,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.84,18.54,4.19,3.60,0.44,NaN,NaN,None,SEA,7.0
6891831,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.74,18.24,3.39,4.45,0.32,NaN,NaN,None,SEA,7.0
6891832,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.66,17.97,2.74,4.55,0.27,NaN,NaN,None,SEA,7.0
6891833,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.59,17.74,2.22,4.87,0.24,NaN,NaN,None,SEA,7.0


In [14]:
locations = locations[~(locations['frameId'] < locations['ball_snap_frame'])]

In [15]:
## Dropping any frames after 3 seconds 
locations['end_frame'] = locations['ball_snap_frame'] + 30

<ipython-input-15-cfefbb3e0bfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['end_frame'] = locations['ball_snap_frame'] + 30


In [16]:
locations = locations[~(locations['frameId'] > locations['end_frame'])]

In [17]:
locations.columns

Index(['gameId_x', 'playId_x', 'playDescription', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'passResult', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'foulName1', 'foulNFLId1',
       'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
       'absoluteYardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInBox', 'personnelD', 'dropBackType', 'pff_playAction',
       'pff_passCoverage', 'pff_passCoverageType', 'uniqueplayId', 'gameId_y',
       'playId_y', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event',
       'homeTeamAbbr', 'ball_snap_frame', 'end_frame'],
      dtype='object')

In [18]:
## Dropping columns not using for analysis
locations = locations[['gameId_x', 'playId_x', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore','homeTeamAbbr',
       'absoluteYardlineNumber', 'pff_passCoverage', 'uniqueplayId', 'nflId', 'frameId', 'time', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'o', 'event',]]

In [19]:
end_time = locations.loc[locations['event'].isin(['pass_forward','autoevent_passforward','qb_sack','qb_strip_sack','handoff'])]
end_time = end_time.groupby('uniqueplayId').agg('min').reset_index()

In [20]:
end_time['end_frame'] = end_time['frameId']
end_time = end_time[['uniqueplayId','end_frame']]

In [21]:
locations = locations.merge(end_time, on = 'uniqueplayId', how = 'left')

In [22]:
locations['end_frame'] = locations['end_frame'].fillna(100)

In [23]:
locations = locations[locations['frameId'] < locations['end_frame']]

In [24]:
locations = locations.drop(['gameId_x','playId_x','end_frame'],axis = 1)

## Adding Scouting Data 

In [25]:
pff['uniqueplayId'] = pff['gameId'].astype(str) + pff['playId'].astype(str)
pff = pff[['uniqueplayId','nflId','pff_positionLinedUp']]

In [26]:
locations = locations.merge(pff, on = ['uniqueplayId','nflId'], how = 'left')
locations['pff_positionLinedUp'] = locations['pff_positionLinedUp'].fillna('football')

In [27]:
locations['pff_positionLinedUp'] = pd.Categorical(locations['pff_positionLinedUp'], positions)
locations = locations.sort_values('pff_positionLinedUp')

In [28]:
football = locations.loc[locations['team'] == 'football',
                               ['uniqueplayId', 'x', 'y']
                               ].rename(columns={'x':'football_x',
                                                 'y':'football_y'})
locations = football.groupby('uniqueplayId').first().merge(locations, on='uniqueplayId', how='right')

In [29]:
locations.tail()

,uniqueplayId,football_x,football_y,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,gameClock,...,time,team,playDirection,x,y,s,a,o,event,pff_positionLinedUp
4340647,20210919081081,68.2,26.6,2,2,12,LV,PIT,40,11:49,...,2021-09-19 17:47:03,football,right,63.34,26.35,0.74,4.46,NaN,None,football
4340648,20210919081081,68.2,26.6,2,2,12,LV,PIT,40,11:49,...,2021-09-19 17:47:03,football,right,63.33,26.27,0.82,4.01,NaN,None,football
4340649,20210919081081,68.2,26.6,2,2,12,LV,PIT,40,11:49,...,2021-09-19 17:47:02,football,right,63.39,26.41,0.89,4.65,NaN,None,football
4340650,20210919081081,68.2,26.6,2,2,12,LV,PIT,40,11:49,...,2021-09-19 17:47:01,football,right,69.39,26.58,6.11,0.52,NaN,autoevent_ballsnap,football
4340651,20210919081081,68.2,26.6,2,2,12,LV,PIT,40,11:49,...,2021-09-19 17:47:01,football,right,70.02,26.58,6.14,0.23,NaN,ball_snap,football


In [30]:
locations.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineNumber',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'homeTeamAbbr',
       'absoluteYardlineNumber', 'pff_passCoverage', 'nflId', 'frameId',
       'time', 'team', 'playDirection', 'x', 'y', 's', 'a', 'o', 'event',
       'pff_positionLinedUp'],
      dtype='object')

In [31]:
locations['new_x'] = abs(locations['x']-locations['football_x'])
locations['new_y'] = np.where(locations['playDirection']=='right', 
                                   locations['football_y']-locations['y'],
                                   locations['y']-locations['football_y'])

In [32]:
locations['Defense'] = np.where(locations['team']== locations['defensiveTeam'], 1, 0)

In [33]:
flipped_o = locations.loc[locations['playDirection'] == 'right',['uniqueplayId','nflId','frameId', 'o']]
flipped_o['o'] = np.where(flipped_o['o'] > 180,
                         flipped_o['o'] - 180,
                         360 - flipped_o['o'])

In [34]:
flipped_o

,uniqueplayId,nflId,frameId,o
0,202109090097,25511.0,6,215.58
1,202109090097,25511.0,31,227.09
2,202109090097,25511.0,32,232.41
3,202109090097,25511.0,33,232.41
4,202109090097,25511.0,34,234.72
...,...,...,...,...
4340647,20210919081081,NaN,25,NaN
4340648,20210919081081,NaN,26,NaN
4340649,20210919081081,NaN,24,NaN
4340650,20210919081081,NaN,7,NaN


In [35]:
locations.update(flipped_o)

In [36]:
locations.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineNumber',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'homeTeamAbbr',
       'absoluteYardlineNumber', 'pff_passCoverage', 'nflId', 'frameId',
       'time', 'team', 'playDirection', 'x', 'y', 's', 'a', 'o', 'event',
       'pff_positionLinedUp', 'new_x', 'new_y', 'Defense'],
      dtype='object')

In [37]:
locations['score_d'] = locations['preSnapHomeScore'] - locations['preSnapVisitorScore']

In [38]:
locations['score_d'] = np.where(locations['defensiveTeam'] == locations['homeTeamAbbr'], locations['score_d'] * 1, locations['score_d'] * -1)

In [39]:
locations = locations.dropna(subset = ['absoluteYardlineNumber'], axis=0)

# Adjancency/Edge Matrices

In [65]:
## Attempting to get the adjacency matrices for all plays
from scipy.spatial.distance import cdist
all_plays  = locations[['nflId','frameId','uniqueplayId','new_x','new_y']].set_index(['nflId','frameId','uniqueplayId'])
dis_matrices = all_plays.groupby(['uniqueplayId','frameId']).apply(lambda x: cdist(x,x,'euclidean'))

In [66]:
binary_adj = dis_matrices.copy()
for frame in range(len(binary_adj)):
  binary_adj[frame] = (binary_adj[frame] < 15).astype(int)

In [67]:
edge_features = dis_matrices.copy()
for frame in range(len(edge_features)):
    edge_features[frame] = np.array([-13*(xi) + 131.0 for xi in edge_features[frame]])

In [68]:
locations.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineNumber',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'homeTeamAbbr',
       'absoluteYardlineNumber', 'pff_passCoverage', 'nflId', 'frameId',
       'time', 'team', 'playDirection', 'x', 'y', 's', 'a', 'o', 'event',
       'pff_positionLinedUp', 'new_x', 'new_y', 'Defense', 'score_d',
       'frames_after_snap'],
      dtype='object')

In [69]:
locations['nflId'] = locations['nflId'].fillna('football')

In [70]:
locations['frames_after_snap'] = locations.groupby(['uniqueplayId', 'nflId']).cumcount()

In [71]:
locations = locations[~(locations['frames_after_snap'] > 30)]

In [72]:
locations['frames_after_snap'].value_counts()

0     163116
1     163116
2     163116
3     163116
4     163116
5     163116
6     163093
7     163093
8     163093
9     163070
10    163024
11    162909
12    162863
13    162587
14    162104
15    160931
16    158953
17    155572
18    150834
19    145038
20    138506
21    132388
22    125649
23    117898
24    109825
25    101361
26     93840
27     85606
28     78798
29     71645
30     65642
Name: frames_after_snap, dtype: int64

In [73]:
node_features = locations[['uniqueplayId', 'quarter',
       'down', 'yardsToGo', 'defensiveTeam',
        'absoluteYardlineNumber','nflId', 
        'frameId', 's', 'o', 'new_x', 'new_y', 'Defense',
       'score_d','frames_after_snap']].set_index(['uniqueplayId','frameId','nflId'])

In [74]:
teams = {'DAL':1, 'TB':2, 'PHI':3, 'ATL':4, 'PIT':5, 'BUF':6, 'CAR':7, 'NYJ':8, 'CIN':9,
       'MIN':10, 'SF':11, 'DET':12, 'JAX':13, 'HOU':14, 'SEA':15, 'IND':16, 'ARI':17, 'TEN':18,
       'WAS':19, 'LAC':20, 'KC':21, 'CLE':22, 'MIA':23, 'NE':24, 'GB':25, 'NO':26, 'DEN':27, 'NYG':28,
       'LA':29, 'CHI':30, 'BAL':31, 'LV':32}

node_features['defensiveTeam'] = node_features.replace({"defensiveTeam": teams})

In [75]:
node_features['o'] = node_features['o'].fillna(0)

In [76]:
node_features[:10]

quarter  down  yardsToGo  defensiveTeam  \
uniqueplayId frameId nflId                                              
202109090097 6.0     25511.0        1     3          2            1.0   
             31.0    25511.0        1     3          2            1.0   
             32.0    25511.0        1     3          2            1.0   
             33.0    25511.0        1     3          2            1.0   
             34.0    25511.0        1     3          2            1.0   
             35.0    25511.0        1     3          2            1.0   
             36.0    25511.0        1     3          2            1.0   
             30.0    25511.0        1     3          2            1.0   
             29.0    25511.0        1     3          2            1.0   
             28.0    25511.0        1     3          2            1.0   

                              absoluteYardlineNumber     s       o  new_x  \
uniqueplayId frameId nflId                                                  
202109090097 6.0     25511.0                    43.0  0.35  215.58   1.38   
             31.0    25511.0                    43.0  1.90  227.09   2.58   
             32.0    25511.0                    43.0  2.18  232.41   2.66   
             33.0    25511.0                    43.0  2.59  232.41   2.73   
             34.0    25511.0                    43.0  2.93  234.72   2.82   
             35.0    25511.0                    43.0  3.41  240.80   2.93   
             36.0    25511.0                    43.0  3.64  249.54   3.05   
             30.0    25511.0                    43.0  1.73  221.76   2.49   
             29.0    25511.0                    43.0  1.42  218.29   2.40   
             28.0    25511.0                    43.0  1.36  213.85   2.29   

                              new_y  Defense  score_d  frames_after_snap  
uniqueplayId frameId nflId                                                
202109090097 6.0     25511.0  -0.27        0        0                  0  
             31.0    25511.0   0.71        0        0                  1  
             32.0    25511.0   0.90        0        0                  2  
             33.0    25511.0   1.14        0        0                  3  
             34.0    25511.0   1.41        0        0                  4  
             35.0    25511.0   1.74        0        0                  5  
             36.0    25511.0   2.07        0        0                  6  
             30.0    25511.0   0.56        0        0                  7  
             29.0    25511.0   0.39        0        0                  8  
             28.0    25511.0   0.29        0        0                  9

In [77]:
y = locations[['uniqueplayId','frameId','nflId','pff_passCoverage']].set_index(['uniqueplayId','frameId','nflId'])

In [78]:
values = {'Cover-0': 0, 'Cover-1': 1, 'Cover-2': 2, 'Cover-3': 3, 'Quarters': 4, '2-Man': 5 ,'Cover-6': 6}
y = y.replace({"pff_passCoverage": values})

In [79]:
enc = OneHotEncoder()
new_y = enc.fit_transform(y)
y = pd.DataFrame.sparse.from_spmatrix(new_y).set_index(y.index)

In [80]:
binary_adj.to_pickle('adj_mat.pkl')

In [81]:
edge_features.to_pickle('edge_features.pkl')

In [82]:
node_features.to_pickle('node_features.pkl')

In [83]:
node_features = pd.read_pickle('node_features.pkl')

In [84]:
node_features = node_features.drop('defensiveTeam', axis = 1)

In [85]:
node_features = (node_features-node_features.mean())/node_features.std()

In [86]:
node_features.to_pickle('node_features.pkl')

In [87]:
y = y.groupby(['uniqueplayId', 'frameId']).nth(0)

In [88]:
y.to_pickle('y.pkl')

In [89]:
edge_features = pd.read_pickle('edge_features.pkl')

In [90]:
def normalize_2d(matrix):
    norm = np.linalg.norm(matrix)
    matrix = matrix/norm  # normalized matrix
    return matrix

In [91]:
for i in range(len(edge_features)):
    edge_features[i] = normalize_2d(edge_features[i])

In [92]:
edge_features.to_pickle('edge_features.pkl')

## Making a simple GNN

In [93]:
import pandas as pd

In [94]:
node_features = pd.read_pickle('node_features.pkl')

In [95]:
simple_nodes = node_features[['new_x', 'new_y', 'Defense', 'o','score_d','frames_after_snap']]

In [96]:
simple_nodes.to_pickle('simple_nodes.pkl')